<a href="https://colab.research.google.com/github/DIPANJAN001/Andrew-Ng-Machine-Learning-Notes/blob/master/SolarPowerManagementSystemFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import random
import numpy as np
import gym
from gym.utils import seeding
from gym.spaces import Space, Discrete, MultiDiscrete,  Box
from gym.spaces.space import Space
import numpy as np
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import metrics

In [87]:
class MyEnv3(gym.Env):
    
      def __init__(self,seed=42):
        np.random.seed(seed)
        self.action_space = Discrete(4)
        self.observation_space=MultiDiscrete([150,130,50])
        self.battery_cap=30
        self.max_battery_discharge=15
        self.max_battery_charge=10
        self.max_battery_capacity=50
        self.state,self.reward, self.done, self.info= None, None, None,None
        
      def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]
      def _render(self):
        txt="state:"+str(self.state)+"reward"+str(self.reward)+"info:"+str(self.info)
        print(txt)

      def _states(self):
        observation_space=[]
        return observation_space
      def _reset(self):
        self.battery_cap=30
        self.state,self.reward,self.done,self.info=[30,0,0],0,False,{}
        return [30,0,0]

      def _step(self,action):
        done=False
        grid_export_=0
        grid_import_=0
        #load=np.random.randint(100,140)
        #pv=np.random.randint(80,130)
        battery_state=self.battery_cap
        reward=0
        data=[]
        if action== 0:
          grid_export_= max(pv-load-self.max_battery_charge,0) # grid export
          self.battery_cap= max(min(self.battery_cap+min(self.max_battery_charge,pv-load),50),0)
          grid_import_=0
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
          }
        if action== 1:
          grid_export_= max(pv-load-self.max_battery_charge,0) # grid export
          self.battery_cap= max(min(self.battery_cap+min(self.max_battery_charge,pv-load),50),0)
          grid_import_=0
          # only battery charge
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
          }
        if action== 2:
            grid_export_=0
            self.battery_cap= min(max(self.battery_cap-self.max_battery_discharge,0),50)# battery discharge
            grid_import_=max(load-pv-self.max_battery_discharge,0)
            info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
             }
        if action== 3:
          grid_export_=0
          self.battery_cap= min(max(self.battery_cap-min(self.max_battery_discharge,load-pv),0),50)#change in battery
          grid_import_=max(load-pv-self.max_battery_discharge,0)
         #only battery discharge
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
                    }     
        
        reward=grid_export_*8-grid_import_*10
        
        if(self.battery_cap<7.0):
              done=True
        #data.append([battery_state,load,pv,action,reward])
        #state.append(load)
        #state.append(pv)
        return reward,self.battery_cap, done,info

In [88]:
'''class Agent:
    def __init__(self,state_size,action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95 #Discount Factor
        self.epsilon = 1.0 # Exploration Rate: How much to act randomly, 
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001 
        self.model = self._create_model()
        
    
    def _create_model(self):
        #Neural Network To Approximate Q-Value function
        model = Sequential()
        model.add(Dense(24,input_dim=self.state_size,activation='relu')) #1st Hidden Layer
        model.add(Dense(24,activation='relu')) #2nd Hidden Layer
        model.add(Dense(self.action_size,activation='linear'))
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self,state,action,reward,done):
        self.memory.append((state,action,reward,done)) #remembering previous experiences
        
    def act(self,state_):
        # Exploration vs Exploitation
        if np.random.rand()<=self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state_) # predict reward value based upon current state
        return np.argmax(act_values[0]) #Left or Right
    
    def train(self,batch_size=32): #method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory,batch_size)
        for state,action,reward,next_state,done in minibatch:
            
            if not done: #boolean 
                target = reward + self.gamma*np.amax(self.model.predict(next_state)[0])
            else:
                target = reward
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state,target_f,epochs=1,verbose=0) #single epoch, x =state, y = target_f, loss--> target_f - 
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def load(self,name):
        self.model.load_weights(name)
    def save(self,name):
        self.model.save_weights(name)'''

"class Agent:\n    def __init__(self,state_size,action_size):\n        self.state_size = state_size\n        self.action_size = action_size\n        self.memory = deque(maxlen=2000)\n        self.gamma = 0.95 #Discount Factor\n        self.epsilon = 1.0 # Exploration Rate: How much to act randomly, \n        self.epsilon_decay = 0.995\n        self.epsilon_min = 0.01\n        self.learning_rate = 0.001 \n        self.model = self._create_model()\n        \n    \n    def _create_model(self):\n        #Neural Network To Approximate Q-Value function\n        model = Sequential()\n        model.add(Dense(24,input_dim=self.state_size,activation='relu')) #1st Hidden Layer\n        model.add(Dense(24,activation='relu')) #2nd Hidden Layer\n        model.add(Dense(self.action_size,activation='linear'))\n        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))\n        return model\n    \n    def remember(self,state,action,reward,done):\n        self.memory.append((state,action,re

In [89]:
env=MyEnv3()

In [90]:
'''agent = Agent(state_size=3,action_size=4)
done = False
state_size = 3
action_size =4
batch_size = 32'''

'agent = Agent(state_size=3,action_size=4)\ndone = False\nstate_size = 3\naction_size =4\nbatch_size = 32'

In [91]:
'''agent = Agent(state_size, action_size) # initialise agent
done = False
for e in range(n_episodes):
    state = env.reset()
    state = np.reshape(state,[1,state_size])
    
    for time in range(5000):
        env.render()
        action = agent.act(state) #action is 0 or 1
        next_state,reward,done,other_info = env.step(action) 
        reward = reward if not done else -10
        next_state = np.reshape(next_state,[1,state_size])
        agent.remember(state,action,reward,next_state,done)
        state = next_state
        
        if done:
            print("Game Episode :{}/{}, High Score:{},Exploration Rate:{:.2}".format(e,n_episodes,time,agent.epsilon))
            break
            
    if len(agent.memory)>batch_size:
        agent.train(batch_size)
    
    if e%50==0:
        agent.save(output_dir+"weights_"+'{:04d}'.format(e)+".hdf5")
        
env.close()'''

'agent = Agent(state_size, action_size) # initialise agent\ndone = False\nfor e in range(n_episodes):\n    state = env.reset()\n    state = np.reshape(state,[1,state_size])\n    \n    for time in range(5000):\n        env.render()\n        action = agent.act(state) #action is 0 or 1\n        next_state,reward,done,other_info = env.step(action) \n        reward = reward if not done else -10\n        next_state = np.reshape(next_state,[1,state_size])\n        agent.remember(state,action,reward,next_state,done)\n        state = next_state\n        \n        if done:\n            print("Game Episode :{}/{}, High Score:{},Exploration Rate:{:.2}".format(e,n_episodes,time,agent.epsilon))\n            break\n            \n    if len(agent.memory)>batch_size:\n        agent.train(batch_size)\n    \n    if e%50==0:\n        agent.save(output_dir+"weights_"+\'{:04d}\'.format(e)+".hdf5")\n        \nenv.close()'

In [92]:
#agent=Agent(3,4)

In [93]:
#state=env._reset()

In [94]:
#state[0]

In [95]:
#print(state)

In [96]:
#state_=[]

In [97]:
'''done=False
for e in range(100):
  score = 0
  #load=np.random.randint(100,140)
  #pv=np.random.randint(80,130)
  state=env._reset()
  done=False
  state_=[]
  state_.append(state[0])
  while not done:
    load=np.random.randint(100,140)
    pv=np.random.randint(80,130)
    state_.append(load)
    state_.append(pv)
    action = env.action_space.sample()
    #action=agent.act(state_)
    reward,observation2, done, info = env._step(action)
    score+=round(reward)
    #agent.remember(state_,action,reward,done)
    state_[0]=[observation2]
  print('episode:{} Profit:{}'.format(e,score))'''
    

"done=False\nfor e in range(100):\n  score = 0\n  #load=np.random.randint(100,140)\n  #pv=np.random.randint(80,130)\n  state=env._reset()\n  done=False\n  state_=[]\n  state_.append(state[0])\n  while not done:\n    load=np.random.randint(100,140)\n    pv=np.random.randint(80,130)\n    state_.append(load)\n    state_.append(pv)\n    action = env.action_space.sample()\n    #action=agent.act(state_)\n    reward,observation2, done, info = env._step(action)\n    score+=round(reward)\n    #agent.remember(state_,action,reward,done)\n    state_[0]=[observation2]\n  print('episode:{} Profit:{}'.format(e,score))"

In [98]:
'''train_start = time.process_time()

nb_episode = 15

for idx, env in enumerate(building_environments):
    print(f"building {idx}")
    for i in range(nb_episode):
        score = 0
        done = False
        observation = env.reset()
        while not done:
            action = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            score -= reward
            agent.store_transition(observation, action, reward, observation_, done)
            agent.train()
            observation = observation_
        print(f"- episode : {i} | score : {score}")
train_end = time.process_time()
train_frugality = train_end - train_start'''

'train_start = time.process_time()\n\nnb_episode = 15\n\nfor idx, env in enumerate(building_environments):\n    print(f"building {idx}")\n    for i in range(nb_episode):\n        score = 0\n        done = False\n        observation = env.reset()\n        while not done:\n            action = agent.choose_action(observation)\n            observation_, reward, done, info = env.step(action)\n            score -= reward\n            agent.store_transition(observation, action, reward, observation_, done)\n            agent.train()\n            observation = observation_\n        print(f"- episode : {i} | score : {score}")\ntrain_end = time.process_time()\ntrain_frugality = train_end - train_start'

In [99]:
'''for e in range(400): #Episode
    #Play 20 episodes 
    score = 0
    observation = env._reset()
    #print(observation)
    done=False
    while not done:
        #env.render()
        action = agent.choose_action(observation)
        #print(action)
        observation1,reward,observation4,done,info = env._step(action)
        print(observation1)
        print(observation4)
        score+=round(reward)
        #print(info['load'],info['power'])
      
    print('episode:{} Profit:{}'.format(e,score))'''

"for e in range(400): #Episode\n    #Play 20 episodes \n    score = 0\n    observation = env._reset()\n    #print(observation)\n    done=False\n    while not done:\n        #env.render()\n        action = agent.choose_action(observation)\n        #print(action)\n        observation1,reward,observation4,done,info = env._step(action)\n        print(observation1)\n        print(observation4)\n        score+=round(reward)\n        #print(info['load'],info['power'])\n      \n    print('episode:{} Profit:{}'.format(e,score))"

In [100]:
import time # Necessary to evaluate frugality
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [101]:
class DeepQL(nn.Module):
    def __init__(self, lr, input_dims, fc1_dims, fc2_dims, nb_actions):
        super(DeepQL, self).__init__()
        self.input_dims = 3
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.nb_actions = 4
        self.fc1 = nn.Linear(self.input_dims, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.fc3 = nn.Linear(self.fc2_dims, self.nb_actions)
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss = nn.MSELoss()
        
        if torch.cuda.is_available():       
            self.device = torch.device("cuda")
        else:
            self.device = torch.device("cpu")
        self.to(self.device)
        
    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        actions = self.fc3(x)
        
        return actions

In [102]:
class Agent():
    def __init__(self, gamma, epsilon, lr, input_dims, batch_size, nb_actions,
                max_mem_size = 100000, eps_end = 0.01, eps_dec = 5e-4):
        
        self.gamma = gamma
        self.epsilon = epsilon
        self.batch_size = batch_size
        self.eps_min = eps_end
        self.eps_dec = eps_dec
        self.lr = lr
        self.action_space = [i for i in range(nb_actions)]
        self.mem_size = max_mem_size
        self.mem_counter = 0
        
        self.Q_eval = DeepQL(self.lr, nb_actions=nb_actions, input_dims=input_dims,
                            fc1_dims=64, fc2_dims=64)
        
        self.state_memory = np.zeros((self.mem_size, input_dims), dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, input_dims), dtype=np.float32)
        
        self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype= np.bool)
    
    def store_transition(self, state, action ,reward, state_, done):
        index = self.mem_counter % self.mem_size
        self.state_memory[index] = state
        self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.new_state_memory[index] = state_
        self.terminal_memory[index] = done
        
        self.mem_counter += 1
    
    def choose_action (self, observation):
        if np.random.random() > self.epsilon:
            state = torch.FloatTensor([observation]).to(self.Q_eval.device)
            actions = self.Q_eval.forward(state)
            action = torch.argmax(actions).item()
        else:
            action = np.random.choice(self.action_space)
        
        return action
    def train(self):
        if self.mem_counter < self.batch_size:
            return
        
        self.Q_eval.optimizer.zero_grad()
        
        max_mem = min(self.mem_counter, self.mem_size)
        batch = np.random.choice(max_mem,self.batch_size, replace=False)
        
        batch_index = np.arange(self.batch_size, dtype=np.int32)
        
        state_batch = torch.tensor(self.state_memory[batch]).to(self.Q_eval.device)
        new_state_batch = torch.tensor(self.new_state_memory[batch]).to(self.Q_eval.device)
        reward_batch = torch.tensor(self.reward_memory[batch]).to(self.Q_eval.device)
        terminal_batch = torch.tensor(self.terminal_memory[batch]).to(self.Q_eval.device)
        
        action_batch = self.action_memory[batch]
        
        q_eval = self.Q_eval.forward(state_batch)[batch_index, action_batch]
        q_next = self.Q_eval.forward(new_state_batch)
        q_next[terminal_batch] = 0.0
        
        q_target = reward_batch + self.gamma * torch.max(q_next,dim=1)[0]
        
        loss = self.Q_eval.loss(q_target, q_eval).to(self.Q_eval.device)
        loss.backward()
        self.Q_eval.optimizer.step()
        
        self.epsilon = self.epsilon - self.eps_dec if self.epsilon > self.eps_min else self.eps_min
    

In [103]:
agent = Agent(gamma=0.99, epsilon=1.0, batch_size=128, nb_actions=4,
             eps_end=0.01, input_dims=3, lr= 0.0001)

<ipython-input-102-7406a06ab070>:23: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.terminal_memory = np.zeros(self.mem_size, dtype= np.bool)


In [108]:
done=False
for e in range(100):
  score = 0
  #load=np.random.randint(100,140)
  #pv=np.random.randint(80,130)
  state=env._reset()
  done=False
  state_=[]
  state_.append(state[0])
  while not done:
    load=np.random.randint(100,140)
    pv=np.random.randint(80,130)
    state_.append(load)
    state_.append(pv)
    print(state_)
    #state1=state_
    #action = env.action_space.sample()
    action=agent.choose_action(state_)
    print(action)
    reward,observation2, done, info = env._step(action)
    score+=round(reward)
    print(observation2)
    state_[0]=observation2
    #agent.store_transition()
  print('episode:{} Profit:{}'.format(e,score))

[30, 110, 128]
3
48
[48, 110, 128, 104, 87]
3
33
[33, 110, 128, 104, 87, 111, 117]
3
39
[39, 110, 128, 104, 87, 111, 117, 104, 109]
2
24
[24, 110, 128, 104, 87, 111, 117, 104, 109, 102, 90]
1
12
[12, 110, 128, 104, 87, 111, 117, 104, 109, 102, 90, 126, 125]
3
11
[11, 110, 128, 104, 87, 111, 117, 104, 109, 102, 90, 126, 125, 126, 121]
1
6
episode:0 Profit:-20
[30, 114, 109]
1
25
[25, 114, 109, 136, 121]
2
10
[10, 114, 109, 136, 121, 122, 85]
3
0
episode:1 Profit:-220
[30, 108, 128]
2
15
[15, 108, 128, 116, 124]
1
23
[23, 108, 128, 116, 124, 103, 84]
0
4
episode:2 Profit:0
[30, 112, 113]
0
31
[31, 112, 113, 122, 82]
3
16
[16, 112, 113, 122, 82, 120, 125]
0
21
[21, 112, 113, 122, 82, 120, 125, 110, 118]
0
29
[29, 112, 113, 122, 82, 120, 125, 110, 118, 127, 103]
0
5
episode:3 Profit:-250
[30, 133, 90]
0
0
episode:4 Profit:0
[30, 116, 114]
3
28
[28, 116, 114, 121, 103]
0
10
[10, 116, 114, 121, 103, 138, 100]
0
0
episode:5 Profit:0
[30, 116, 116]
2
15
[15, 116, 116, 126, 113]
0
2
episode:6 P